# IN2110 Oblig 2a
## Oppgave 2 og 3
Etter at du har trent modellen fra kommandolinja, gjøres resten av programmeringen, rapporteringen og besvarelse ellers i dette dokumentet. Lag celler for markdown og kode etter behov.

In [24]:
import spacy
from helpers.conllu import ConlluDoc
print("spaCy version:",spacy.__version__) 


spaCy version: 3.7.2


In [25]:
nb = spacy.load("output/model-best")
# lese inn
conllu_dev = ConlluDoc.from_file("no_bokmaal-ud-dev.conllu")
# konvertere til spaCy-objekter, gull-dataene for dev-settet vårt
# liste med doc-objekter som hver representerer en setning 
# hvert doc-objekt har en liste med hode, indeks og dependensrelasjon (head, i, dep_)
dev_docs = conllu_dev.to_spacy(nb)
# versjon uten labels
dev_docs_unlabeled = conllu_dev.to_spacy(nb, keep_labels=False)

In [26]:
# For å se hvordan dokumentene ser ut
print(dev_docs[0])
if dev_docs:
    første_doc = dev_docs[0]
    tokens_i_første_doc = list(første_doc)  # Konverterer til en liste for enkelhets skyld

    print(f"Antall tokens i første setning: {len(tokens_i_første_doc)}")
    for token in tokens_i_første_doc:
        print(f"Tekst: {token.text}, Indeks: {token.i}, Hode: {token.head.text}, Dependensrelasjon: {token.dep_}")
else:
    print("dev_docs er tom.")

Dommer Finn Eilertsen avstår, selvfølgelig bevisst, fra å «sette ord på» det inntrykk retten for sitt vedkommende måtte ha dannet seg av de handlinger retten finner bevist og av lovovertrederen. 
Antall tokens i første setning: 35
Tekst: Dommer, Indeks: 0, Hode: Finn, Dependensrelasjon: nmod
Tekst: Finn, Indeks: 1, Hode: avstår, Dependensrelasjon: nsubj
Tekst: Eilertsen, Indeks: 2, Hode: Finn, Dependensrelasjon: flat:name
Tekst: avstår, Indeks: 3, Hode: avstår, Dependensrelasjon: root
Tekst: ,, Indeks: 4, Hode: avstår, Dependensrelasjon: punct
Tekst: selvfølgelig, Indeks: 5, Hode: bevisst, Dependensrelasjon: advmod
Tekst: bevisst, Indeks: 6, Hode: avstår, Dependensrelasjon: advmod
Tekst: ,, Indeks: 7, Hode: bevisst, Dependensrelasjon: punct
Tekst: fra, Indeks: 8, Hode: sette, Dependensrelasjon: mark
Tekst: å, Indeks: 9, Hode: sette, Dependensrelasjon: mark
Tekst: «, Indeks: 10, Hode: sette, Dependensrelasjon: punct
Tekst: sette, Indeks: 11, Hode: avstår, Dependensrelasjon: advcl
Tekst:

In [27]:
doc = dev_docs_unlabeled[5] 
doc
#doc.has_annotation("DEP")
#doc = nb(doc)
#doc.has_annotation("DEP")

Thomassen er på vei til sin neste gjerning. 

In [28]:
# funksjon som tar imot en liste av spacy doc-objekter og parser disse
# returnerer en liste med de parsede objektene, disse blir prediksjonene? 
def parse_docs(spacy_list):
    parsed_docs = []
    for doc in spacy_list:
        parsed_docs.append(nb(doc))
        
    return parsed_docs

#parse_docs(dev_docs_unlabeled)



In [36]:

def attachment_score(true, pred):
    """Tar inn to lister av doc der liste nr 2 har predikert depedens. Finner andel korrekte head (UAS), og korrekte head+dependensrelasjoner (LAS)
    Merk at depedensrelasjonene som SpaCy estimerer har "ROOT" med store bokstaver, mens våre data har "root" med små.
    Dette er den samme relasjonen, og skal teller som korrekt estimert """
    #TODO
    correct_head = 0
    correct_deprel = 0
    total_tokens = 0

    #sjekke at de er like lange 
    if len(true) != len(pred):
        raise ValueError("Listene 'true' og 'pred' må ha samme lengde (samme antall dokumenter).")


    for gold_doc, pred_doc in zip(true, pred): 
        if len(gold_doc) != len(pred_doc):
            print(f"Advarsel: Antall tokens er forskjellig i gullstandarddokumentet og det predikerte dokumentet. Hopper over dette dokumentet.")
            continue

        for gold_token, pred_token in zip(gold_doc, pred_doc):
            total_tokens += 1
            if gold_token.head.i == pred_token.head.i: 
                correct_head += 1

                #dependensrelasjon 
                gold_dep = gold_token.dep_.lower()
                pred_dep = pred_token.dep_.lower()

                if gold_dep == pred_dep:
                    correct_deprel += 1
    print("Correct head count: ", correct_head)
    print("Correct deprel: ", correct_deprel)
    print("Number tokens: ", total_tokens)

    uas = correct_head / total_tokens if total_tokens > 0 else 0.0
    # kun correct_deprel siden vi sjekker dette når vi sjekker korrekte hoder 
    las = correct_deprel / total_tokens if total_tokens > 0 else 0.0

    return uas, las


In [30]:
#nynorsk trebank
conllu_nynorsk = ConlluDoc.from_file("no_nynorsk-ud-dev.conllu")
nn_docs = conllu_nynorsk.to_spacy(nb)
nn_docs_unlabeled = conllu_nynorsk.to_spacy(nb, keep_labels=False)

#nynorsklia trebank 
conllu_lia = ConlluDoc.from_file("no_nynorsklia-ud-dev.conllu")
lia_docs = conllu_lia.to_spacy(nb)
lia_docs_unlabeled = conllu_lia.to_spacy(nb, keep_labels=False)

In [37]:
parsed = parse_docs(dev_docs_unlabeled)
parsed_nn = parse_docs(nn_docs_unlabeled)
parsed_lia = parse_docs(lia_docs_unlabeled)

print("\n-------- BOKMAAL --------")
uas, las = attachment_score(dev_docs, parsed)
print("UAS for bokmaal: ", uas)
print("LAS for bokmaal: ", las)

print("\n-------- NYNORSK --------")
nn_uas, nn_las = attachment_score(nn_docs, nn_docs_unlabeled)
print("UAS for nynorsk: ", nn_uas)
print("LAS for nynorsk: ", nn_las)

print("\n-------- TALESPRAAK --------")
lia_uas, lia_las = attachment_score(lia_docs, lia_docs_unlabeled)
print("UAS for talespraak: ", lia_uas)
print("LAS for talespraak: ", lia_las)




-------- BOKMAAL --------
Correct head count:  31912
Correct deprel:  30745
Number tokens:  36369
UAS for bokmaal:  0.8774505760400341
LAS for bokmaal:  0.8453628089856746

-------- NYNORSK --------
Correct head count:  20395
Correct deprel:  17879
Number tokens:  31250
UAS for nynorsk:  0.65264
LAS for nynorsk:  0.572128

-------- TALESPRAAK --------
Correct head count:  4660
Correct deprel:  3565
Number tokens:  10163
UAS for talespraak:  0.45852602577978946
LAS for talespraak:  0.350782249335826


| Spraak     |   UAS     |   LAS    |
| ---------  | --------- | ---------|
| Bokmaal    | 0.877     | 0.845    |
| Nynorsk    | 0.652     | 0.572    |
| Talespraak | 0.458     | 0.350    |


Her ser vi at vi får betydelig høyere score på bokmål-trebanken enn de to andre. Nå vi ser på antall ord i trebankene, ser vi også at bokmål og nynorsk har betydelig mer data enn talespråk/LIA-banken. Dette betyr at den har mindre data å trene på, og kan derfor være grunnen til at den får så lav score sammenlignet med de andre. Det er også naturlig at LAS får lavere score enn UAS, fordi i LAS så deler vi med en lavere tall. 